In [1]:
#load up the basics
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd

#Set GPU to use
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#import TF stuff
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Check TensorFlow Version
print('TF version: {}' .format(tf.__version__))

#Check for GPU utilization
if tf.test.gpu_device_name():
    print(' GPU: {}'.format(tf.test.gpu_device_name()))
else:
    print("No GPU")

TF version: 2.9.1
 GPU: /device:GPU:0


In [2]:
#Load up the TF model

#model path
model_path = './model/FRF_mar15_remap_fullmodel_model'

# load model into tf
model = tf.keras.models.load_model(model_path, compile = True)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 7)  91          ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 7)  28         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                              

 activation_6 (Activation)      (None, 128, 128, 28  0           ['batch_normalization_6[0][0]']  
                                )                                                                 
                                                                                                  
 activation_7 (Activation)      (None, 128, 128, 28  0           ['batch_normalization_7[0][0]']  
                                )                                                                 
                                                                                                  
 conv2d_7 (Conv2D)              (None, 128, 128, 28  63532       ['activation_6[0][0]']           
                                )                                                                 
                                                                                                  
 add_2 (Add)                    (None, 128, 128, 28  0           ['activation_7[0][0]',           
          

                                                                                                  
 activation_15 (Activation)     (None, 32, 32, 112)  0           ['batch_normalization_15[0][0]'] 
                                                                                                  
 conv2d_16 (Conv2D)             (None, 32, 32, 112)  1016176     ['activation_15[0][0]']          
                                                                                                  
 up_sampling2d (UpSampling2D)   (None, 64, 64, 112)  0           ['conv2d_16[0][0]']              
                                                                                                  
 concatenate (Concatenate)      (None, 64, 64, 168)  0           ['up_sampling2d[0][0]',          
                                                                  'add_3[0][0]']                  
                                                                                                  
 batch_nor

                                )                                                                 
                                                                                                  
 conv2d_25 (Conv2D)             (None, 256, 256, 28  1988        ['concatenate_2[0][0]']          
                                )                                                                 
                                                                                                  
 batch_normalization_23 (BatchN  (None, 256, 256, 28  112        ['conv2d_23[0][0]']              
 ormalization)                  )                                                                 
                                                                                                  
 batch_normalization_24 (BatchN  (None, 256, 256, 28  112        ['conv2d_25[0][0]']              
 ormalization)                  )                                                                 
          

In [3]:
#define the number of classes of the image...

num_classes = 4

#this is a fn to make sure the doodler-derived labels go from 0- (num_classes-1)

def D_Filter(image):
    image[image>(num_classes-1)] = (num_classes-1)
    return image

def standardize(img):
    #standardization using adjusted standard deviation

    N = np.shape(img)[0] * np.shape(img)[1]
    s = np.maximum(np.std(img), 1.0/np.sqrt(N))
    m = np.mean(img)
    img = (img - m) / s
    del m, s, N
    #
    if np.ndim(img)==2:
        img = np.dstack((img,img,img))

    return img


In [4]:
#test the model

#define test directory
test_dir_im = '../data/c6_test/images'
test_dir_lab = '../data/c6_test/labels'

#load images
i_test = []
i_images = []
for img in os.listdir(test_dir_im):
    #pull the name of the image
    print(img)
    i_images.append(img)
    #pull the image
    img = os.path.join(test_dir_im, img)
    img = tf.keras.preprocessing.image.load_img(img,target_size = (512,512))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = standardize(img)
    
    img = np.expand_dims(img,axis=0)
#     i_test = img
    
    i_test.append(img)
    
#load labels
l_test = []
l_images = []
for img in os.listdir(test_dir_lab):
    #pull the name of the image
    l_images.append(img)
    #pull the image
    img = os.path.join(test_dir_lab, img)
    img = tf.keras.preprocessing.image.load_img(img,target_size = (512,512),color_mode = "grayscale")
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = D_Filter(img)
    img = tf.keras.utils.to_categorical(img, num_classes=4)
    
    img = np.expand_dims(img,axis=0)
#     l_test = img
    
    l_test.append(img)
    
#stack
i_test = np.vstack(i_test)
l_test = np.vstack(l_test)

FRF_c6_snap_20160601_EBG.jpg
FRF_c6_snap_20210101_EBG.jpg
FRF_c6_snap_20170101_EBG.jpg
FRF_c6_snap_20160801_EBG.jpg
FRF_c6_snap_20170601_EBG.jpg
FRF_c6_snap_20200101_EBG.jpg
FRF_c6_snap_20181201_EBG.jpg
FRF_c6_snap_20170501_EBG.jpg
FRF_c6_snap_20170301_EBG.jpg
FRF_c6_snap_20181101_EBG.jpg


In [5]:
np.shape(i_test)

(10, 512, 512, 3)

In [6]:
#Doodleverse metrics, 
# https://github.com/Doodleverse/doodleverse_utils/blob/main/doodleverse_utils/model_imports.py

def mean_iou(y_true, y_pred):
    """
    mean_iou(y_true, y_pred)
    This function computes the mean IoU between `y_true` and `y_pred`: this version is tensorflow (not numpy) 
    and is used by tensorflow training and evaluation functions
    INPUTS:
        * y_true: true masks, one-hot encoded.
            * Inputs are B*W*H*N tensors, with
                B = batch size,
                W = width,
                H = height,
                N = number of classes
        * y_pred: predicted masks, either softmax outputs, or one-hot encoded.
            * Inputs are B*W*H*N tensors, with
                B = batch size,
                W = width,
                H = height,
                N = number of classes
    OPTIONAL INPUTS: None
    GLOBAL INPUTS: None
    OUTPUTS:
        * IoU score [tensor]
    """
    yt0 = y_true[:, :, :, 0]
    yp0 = tf.keras.backend.cast(y_pred[:, :, :, 0] > 0.5, "float32")
    inter = tf.math.count_nonzero(tf.logical_and(tf.equal(yt0, 1), tf.equal(yp0, 1)))
    union = tf.math.count_nonzero(tf.add(yt0, yp0))
    iou = tf.where(tf.equal(union, 0), 1.0, tf.cast(inter / union, "float32"))
    return iou


# -----------------------------------
def dice_coef(y_true, y_pred):
    """
    dice_coef(y_true, y_pred)
    This function computes the mean Dice coefficient between `y_true` and `y_pred`: this version is 
    tensorflow (not numpy) and is used by tensorflow training and evaluation functions
    INPUTS:
        * y_true: true masks, one-hot encoded.
            * Inputs are B*W*H*N tensors, with
                B = batch size,
                W = width,
                H = height,
                N = number of classes
        * y_pred: predicted masks, either softmax outputs, or one-hot encoded.
            * Inputs are B*W*H*N tensors, with
                B = batch size,
                W = width,
                H = height,
                N = number of classes
    OPTIONAL INPUTS: None
    GLOBAL INPUTS: None
    OUTPUTS:
        * Dice score [tensor]
    """
    smooth = 1.0
    y_true_f = tf.reshape(tf.dtypes.cast(y_true, tf.float32), [-1])
    y_pred_f = tf.reshape(tf.dtypes.cast(y_pred, tf.float32), [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2.0 * intersection + smooth) / (
        tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth
    )


In [7]:
model.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy',
             metrics = [mean_iou, dice_coef])

In [8]:

#lets now train the model! We can store all teh model output in 'history'.
# note that we are using the training generator for training data, the val 
# generator for validation data, we set the model to train for some number of epochs, 
# we tell teh network how many steps there are to get through the entire data set
# and we also give the network our callback to use.

results = model.evaluate(i_test,l_test)

1/1 [==============================] - 8s 8s/step - loss: 0.8040 - mean_iou: 0.7962 - dice_coef: 0.8011
